<a href="https://colab.research.google.com/github/wujj0326/COVID_Risk_Competition/blob/master/Get_risk_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LA County Risk Score

## import risk data

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [34]:
risk = pd.read_csv('https://raw.githubusercontent.com/wujj0326/COVID_Risk_Competition/master/Data/risk_confirmed.csv')

In [35]:
risk.head()

,City,confirmed_cases,death_cases,pop2010,avg_traffic,avg_asthma,avg_cardiovascular,avg_poverty,avg_elderly,pop_dens
0,Acton,11,1,53654,1058.69,73.62,11.00,70.81,5.24,120974.0
1,Agoura Hills,35,1,60009,1023.45,77.60,10.95,72.49,5.45,177875.5
2,Alhambra,226,8,112060,1816.86,47.43,7.17,46.00,10.40,551582.0
3,Altadena,157,5,69531,1379.59,34.64,5.34,55.34,10.67,864902.9
4,Arcadia,104,7,70416,1305.56,49.61,6.12,73.43,8.78,755363.9


In [10]:
# normalize the scale by using minmaxscaler
scale_risk = MinMaxScaler(feature_range=(0, 1))
features = scale_risk.fit_transform(risk.iloc[:,3:-1])

#  vulnerable factors related to death cases

# •       elderly

# •       asthma

# •       cardiovascular

#  vulnearble factors related to infected cases

# •       poverty: the higher the value the poorer the area

# •       traffic

# •       population

confirm_feat = np.sum(features[:,np.r_[0,1,4]],axis=1)
death_feat = np.sum(features[:,np.r_[2,3,5]],axis=1)

In [11]:
# apply sigmoid function to confirm feature and death feature
sig_confirmed = 1/(1 + np.exp(confirm_feat)) 
sig_death = 1/(1 + np.exp(death_feat)) 

In [12]:
risk['confirmed_score'] = sig_confirmed
risk['death_score'] = sig_death

In [13]:
risk_score = risk.iloc[:,np.r_[0,-2,-1]]

## import the predicted data

In [32]:
case_fit = pd.read_csv('https://raw.githubusercontent.com/wujj0326/COVID_Risk_Competition/master/Data/cases_prediction.csv')

In [33]:
case_fit

,time,positive,positive_low,positive_high
0,7/29/2020,181121.1035,179840.8907,182761.2016
1,7/30/2020,183586.7964,182180.9785,185600.7594
2,7/31/2020,186078.0061,184542.7771,188478.2714
3,8/1/2020,188595.2395,186926.7146,191394.5058
4,8/2/2020,191139.0128,189333.2267,194350.2463
5,8/3/2020,193709.8516,191762.7570,197346.2921
6,8/4/2020,196308.2903,194215.7553,200383.4582


In [30]:
death = pd.read_csv('https://raw.githubusercontent.com/wujj0326/COVID_Risk_Competition/master/Data/deaths_prediction.csv')

In [31]:
death

,time,dead,dead_low,dead_high
0,7/29/2020,4457.672053,4442.130056,4472.148723
1,7/30/2020,4489.281939,4470.465939,4506.684092
2,7/31/2020,4520.776952,4498.364630,4541.215381
3,8/1/2020,4552.160918,4526.103289,4575.747185
4,8/2/2020,4583.437635,4553.685923,4610.284097
5,8/3/2020,4614.610869,4581.116484,4644.830712
6,8/4/2020,4645.684357,4608.398865,4679.391622


## Get Hazard Score


In [18]:
prediction = pd.merge(case_fit, death, left_on='time', right_on='time')

In [19]:
prediction = prediction.iloc[:,[0,1,4]]

In [20]:
prediction['positive_increase'] = prediction['positive'].diff()
prediction['dead_increase'] = prediction['dead'].diff()

In [21]:
prediction = prediction.iloc[1:,[0,3,4]].reset_index(drop=True)

In [22]:
def norm_hazard(alpha,data):
  alpha = alpha
  data['hazard'] = data['positive_increase']*alpha + data['dead_increase']*(1-alpha)
  return data

In [23]:
pred_haz = norm_hazard(0.012,prediction).iloc[:,[0,-1]]

## Get Risk Score

In [29]:
risk_score['risk_{}'.format(pred_haz['time'][0])] = risk_score['confirmed_score']*pred_haz['hazard'][0]
risk_score['risk_{}'.format(pred_haz['time'][1])] = risk_score['confirmed_score']*pred_haz['hazard'][1]
risk_score['risk_{}'.format(pred_haz['time'][2])] = risk_score['confirmed_score']*pred_haz['hazard'][2]
risk_score['risk_{}'.format(pred_haz['time'][3])] = risk_score['confirmed_score']*pred_haz['hazard'][3]
risk_score['risk_{}'.format(pred_haz['time'][4])] = risk_score['confirmed_score']*pred_haz['hazard'][4]
risk_score['risk_{}'.format(pred_haz['time'][5])] = risk_score['confirmed_score']*pred_haz['hazard'][5]

,City,confirmed_score,death_score,risk_7/30/2020,risk_7/31/2020,risk_8/1/2020,risk_8/2/2020,risk_8/3/2020,risk_8/4/2020,weekly_score
0,Acton,0.189272,0.220192,11.511339,11.547813,11.586154,11.626378,11.668499,11.712534,11.608786
1,Agoura Hills,0.181774,0.213950,11.055304,11.090333,11.127155,11.165785,11.206238,11.248528,11.148890
2,Alhambra,0.144250,0.270825,8.773122,8.800920,8.830141,8.860796,8.892898,8.926458,8.847389
3,Altadena,0.187301,0.329177,11.391408,11.427503,11.465444,11.505249,11.546931,11.590507,11.487840
4,Arcadia,0.155135,0.314134,9.435108,9.465004,9.496429,9.529398,9.563922,9.600014,9.514979
...,...,...,...,...,...,...,...,...,...,...
85,Venice,0.341497,0.290481,20.769475,20.835284,20.904461,20.977035,21.053033,21.132483,20.945295
86,West Hills,0.418424,0.205003,25.448053,25.528687,25.613447,25.702369,25.795487,25.892833,25.663479
87,Wilmington,0.234219,0.206903,14.244955,14.290090,14.337536,14.387312,14.439436,14.493927,14.365543
88,Winnetka,0.254805,0.189505,15.496942,15.546045,15.597661,15.651811,15.708517,15.767797,15.628129


In [ ]:
risk_score['weekly_score'] = risk_score.iloc[:,3:].mean(axis=1)

In [40]:
level = []
for risk in risk_score['weekly_score']:
  if risk <= risk_score['weekly_score'].quantile(0.25):
    b = 1
  elif risk <= risk_score['weekly_score'].quantile(0.50) and risk >risk_score['weekly_score'].quantile(0.25):
    b = 2 
  elif risk <= risk_score['weekly_score'].quantile(0.75) and risk > risk_score['weekly_score'].quantile(0.50):
    b = 3
  else: b =4
  level.append(b)

In [41]:
risk_score['level'] = level

In [43]:
risk_trunc = risk_score.iloc[:,[0,3,4,5,6,7,8,9,10]]

,City,risk_7/30/2020,risk_7/31/2020,risk_8/1/2020,risk_8/2/2020,risk_8/3/2020,risk_8/4/2020,weekly_score,level
0,Acton,11.511339,11.547813,11.586154,11.626378,11.668499,11.712534,11.608786,2
1,Agoura Hills,11.055304,11.090333,11.127155,11.165785,11.206238,11.248528,11.148890,1
2,Alhambra,8.773122,8.800920,8.830141,8.860796,8.892898,8.926458,8.847389,1
3,Altadena,11.391408,11.427503,11.465444,11.505249,11.546931,11.590507,11.487840,2
4,Arcadia,9.435108,9.465004,9.496429,9.529398,9.563922,9.600014,9.514979,1
...,...,...,...,...,...,...,...,...,...
85,Venice,20.769475,20.835284,20.904461,20.977035,21.053033,21.132483,20.945295,4
86,West Hills,25.448053,25.528687,25.613447,25.702369,25.795487,25.892833,25.663479,4
87,Wilmington,14.244955,14.290090,14.337536,14.387312,14.439436,14.493927,14.365543,2
88,Winnetka,15.496942,15.546045,15.597661,15.651811,15.708517,15.767797,15.628129,3


In [44]:
risk_trunc.to_csv('final_risk_score.csv',index=False)